In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
from helper_functions import non_max_suppression_fast

ModuleNotFoundError: No module named 'helper_functions'

In [5]:
def detect(image, base_model_name):
    
#     load the model
    model = load_model(base_model_name)
    
    
#     Perform Selective Search for the given image and then passes each region through the CNN.
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    results = ss.process()
    
    copy = image.copy()
    copy2 = image.copy()
    
    positive_boxes = []
    probs = []

#     extract the x, y, w, h co ordinates of the bounding box of the region
    for box in results:
        x1 = box[0]
        y1 = box[1]
        x2 = box[0] + box[2]
        y2 = box[1] + box[3]

#     obtain the region of interest with these co ordinates and resize, reshape it
        roi = image.copy()[y1:y2, x1:x2]
        roi = cv2.resize(roi, (128, 128))
        roi_use = roi.reshape((1, 128, 128, 3))
        
#     predict the class of extracted roi as plate or not_plate
        class_pred = model.predict_classes(roi_use)[0][0]

        if class_pred == 1:    # if predicted class is plate
            prob = model.predict(roi_use)[0][0]
            
            if prob > 0.98:    # CNN predicts with more than 98% confidence that region has number plate
                positive_boxes.append([x1, y1, x2, y2])
                probs.append(prob)
                cv2.rectangle(copy2, (x1, y1), (x2, y2), (255, 0, 0), 5)

#     selecting the required and accurate box among all
#     selecting a single entity out of many overlapping entities using non_max_suppression_fast 
    cleaned_boxes = non_max_suppression_fast(np.array(positive_boxes), 0.1, probs)
    total_boxes = 0
    for clean_box in cleaned_boxes:
        clean_x1 = clean_box[0]
        clean_y1 = clean_box[1]
        clean_x2 = clean_box[2]
        clean_y2 = clean_box[3]
        total_boxes += 1
        cv2.rectangle(copy, (clean_x1, clean_y1), (clean_x2, clean_y2), (0, 255, 0), 3)

    plt.imshow(copy)
    plt.show()

In [6]:
# function to detect number plate for given image
def play(image_name, model_name):
    test_img = cv2.imread(image_name)
    detect(test_img, model_name)

In [5]:
# testing the model for different images

In [8]:
play('1.jpg', 'base_model.h5')

OSError: SavedModel file does not exist at: base_model.h5\{saved_model.pbtxt|saved_model.pb}

In [7]:
play('2.jpg', 'base_model.h5')

NameError: name 'load_model' is not defined

In [8]:
play('3.jpg', 'base_model.h5')

NameError: name 'load_model' is not defined

In [9]:
play('4.jpg', 'base_model.h5') 

NameError: name 'load_model' is not defined

In [10]:
play('5.jpg', 'base_model.h5')

NameError: name 'load_model' is not defined

In [11]:
play('many_plates.jpg','base_model.h5')

NameError: name 'load_model' is not defined

In [14]:
play('6.jpeg','base_model.h5')

NameError: name 'load_model' is not defined

In [2]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


'c:\users\om' is not recognized as an internal or external command,
operable program or batch file.
